In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
hnm_file_format = 'data/HnM_{}.parquet'

In [3]:
cats = pd.read_parquet(hnm_file_format.format('Categories'))
filters = pd.read_parquet(hnm_file_format.format('Filters'))
items = pd.read_parquet(hnm_file_format.format('Items'))
attributes = pd.read_parquet(hnm_file_format.format('FilterAttributes'))

In [4]:
cats[(cats['Parent_Category'].isna()) & (cats['Skip'] == False)].groupby(['Top_Category'])['Category'].count().reset_index()

,Top_Category,Category
0,Baby,5
1,Kids,6
2,Men,18
3,Women,23


In [5]:
cats.head()

,Category,URL,Parent_Category,Top_Category,Skip
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False


In [6]:
cats[~cats['Parent_Category'].isnull()]

,Category,URL,Parent_Category,Top_Category,Skip
29,Cut-Out-Tops,/de_de/damen/produkte/tops/cut-out.html,0.0,Women,False
30,Bralette-Tops,/de_de/damen/produkte/tops/bralette.html,0.0,Women,False
31,Bandeau-Tops,/de_de/damen/produkte/tops/bandeau.html,0.0,Women,False
32,Oberteile mit Puffärmeln,/de_de/damen/produkte/tops/puffaermel.html,0.0,Women,False
33,Trägertops & Tanktops,/de_de/damen/produkte/tops/westen.html,0.0,Women,False
...,...,...,...,...,...
402,Ballerinas,/de_de/kinder/produkte/schuhe/ballerinas.html,370.0,Kids,False
403,Stiefel,/de_de/kinder/produkte/schuhe/stiefel.html,370.0,Kids,False
404,Gummistiefel,/de_de/kinder/produkte/schuhe/gummistiefel.html,370.0,Kids,False
405,Sandalen & Flip-Flops,/de_de/kinder/produkte/schuhe/sandalen-flip-fl...,370.0,Kids,False


### Creating category name which is concatenated Top Category and Category Name (for parent categories) or top_category, parent_category and category (for  child categories)

In [7]:
cats.merge(
    cats[cats['Parent_Category'].isnull()], 
    right_index=True, 
    left_on='Parent_Category', 
    how='left'
)

,Category_x,URL_x,Parent_Category_x,Top_Category_x,Skip_x,Category_y,URL_y,Parent_Category_y,Top_Category_y,Skip_y
0,Shirts & Tops,/de_de/damen/produkte/tops.html,NaN,Women,False,NaN,NaN,NaN,NaN,NaN
1,Basics,/de_de/damen/produkte/basics.html,NaN,Women,False,NaN,NaN,NaN,NaN,NaN
2,Hemden & Blusen,/de_de/damen/produkte/hemden-blusen.html,NaN,Women,False,NaN,NaN,NaN,NaN,NaN
3,Cardigans & Pullover,/de_de/damen/produkte/cardigans-pullover.html,NaN,Women,False,NaN,NaN,NaN,NaN,NaN
4,Strickwaren,/de_de/damen/produkte/strickwaren.html,NaN,Women,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
402,Ballerinas,/de_de/kinder/produkte/schuhe/ballerinas.html,370.0,Kids,False,Schuhe,/de_de/kinder/produkte/schuhe.html,NaN,Kids,False
403,Stiefel,/de_de/kinder/produkte/schuhe/stiefel.html,370.0,Kids,False,Schuhe,/de_de/kinder/produkte/schuhe.html,NaN,Kids,False
404,Gummistiefel,/de_de/kinder/produkte/schuhe/gummistiefel.html,370.0,Kids,False,Schuhe,/de_de/kinder/produkte/schuhe.html,NaN,Kids,False
405,Sandalen & Flip-Flops,/de_de/kinder/produkte/schuhe/sandalen-flip-fl...,370.0,Kids,False,Schuhe,/de_de/kinder/produkte/schuhe.html,NaN,Kids,False


In [8]:
def apply_name(row):
    if math.isnan(row['Parent_Category_x']):
        return '_'.join([row['Top_Category_x'], row['Category_x'].replace(' ', '-')])
    else:
        return '_'.join([row['Top_Category_x'], row['Category_y'].replace(' ', '-'), row['Category_x'].replace(' ', '-')])

cats['Category_Name'] = cats.merge(
    cats[cats['Parent_Category'].isnull()], 
    right_index=True, 
    left_on='Parent_Category', 
    how='left'
).apply(apply_name, axis=1)

In [9]:
cats[cats['Parent_Category'].isnull()]['Category_Name'].shape

(52,)

In [10]:
# adding top category name and parent category id
items = items.merge(
    cats[['Category', 'Parent_Category', 'Top_Category', 'Category_Name']], 
    how='left', 
    left_on='Category', 
    right_index=True
).drop(
    columns='Category_y'
).rename(
    columns={
        'Category_x': 'Category',
        'Category_Name': 'Sub_Category_Name'
    }
)

In [11]:
items.head()

,Code,Name,URL,Img_URL,Img_Ext,Category,Parent_Category,Top_Category,Sub_Category_Name
0,1071498001,Cut-out-Shirt,/de_de/productpage.1071498001.html,//lp2.hm.com/hmgoepprod?set=source[/a2/90/a290...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops
1,1069183001,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183001.html,//lp2.hm.com/hmgoepprod?set=source[/12/09/1209...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops
2,0985757001,Kurzshirt mit Cut-out,/de_de/productpage.0985757001.html,//lp2.hm.com/hmgoepprod?set=source[/68/a3/68a3...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops
3,0985757002,Kurzshirt mit Cut-out,/de_de/productpage.0985757002.html,//lp2.hm.com/hmgoepprod?set=source[/10/22/1022...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops
4,1069183002,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183002.html,//lp2.hm.com/hmgoepprod?set=source[/d1/3a/d13a...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops


In [12]:
# adding name of parent category also
items = items.merge(
    cats[['Category', 'Parent_Category', 'Category_Name']], 
    left_on='Parent_Category', 
    right_index=True, 
    how='left'
).rename(
    columns={
        'Category_x': 'Category',
        'Category_y': 'Parent_Category_Name',
        'Parent_Category_x': 'Parent_Category'
}
).drop(
    columns='Parent_Category_y'
)

In [13]:
items.head()

,Code,Name,URL,Img_URL,Img_Ext,Category,Parent_Category,Top_Category,Sub_Category_Name,Parent_Category_Name,Category_Name
0,1071498001,Cut-out-Shirt,/de_de/productpage.1071498001.html,//lp2.hm.com/hmgoepprod?set=source[/a2/90/a290...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops,Shirts & Tops,Women_Shirts-&-Tops
1,1069183001,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183001.html,//lp2.hm.com/hmgoepprod?set=source[/12/09/1209...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops,Shirts & Tops,Women_Shirts-&-Tops
2,0985757001,Kurzshirt mit Cut-out,/de_de/productpage.0985757001.html,//lp2.hm.com/hmgoepprod?set=source[/68/a3/68a3...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops,Shirts & Tops,Women_Shirts-&-Tops
3,0985757002,Kurzshirt mit Cut-out,/de_de/productpage.0985757002.html,//lp2.hm.com/hmgoepprod?set=source[/10/22/1022...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops,Shirts & Tops,Women_Shirts-&-Tops
4,1069183002,Kurzes Shirt mit Cut-out,/de_de/productpage.1069183002.html,//lp2.hm.com/hmgoepprod?set=source[/d1/3a/d13a...,.jpg,29.0,0.0,Women,Women_Shirts-&-Tops_Cut-Out-Tops,Shirts & Tops,Women_Shirts-&-Tops


In [14]:
items.groupby('Category_Name')['Code'].agg(['count', pd.Series.nunique])

,count,nunique
Category_Name,,
Baby_Accessoires,136,125
Baby_Kleidung,1568,1075
Baby_Outdoor,71,63
Baby_Schuhe,68,68
Kids_Accessoires,309,296
Kids_Kleidung,5031,4249
Kids_Outdoor,323,295
Kids_Schuhe,215,215
Men_Accessoires,337,320


In [15]:
items[items['Category_Name'] == 'Men_Shorts']['Code'].value_counts()

0970367001    3
0884754012    2
1035209003    2
1035209002    2
0884754007    2
             ..
0986789002    1
1045432003    1
1045432002    1
0986789004    1
1037875006    1
Name: Code, Length: 76, dtype: int64

In [16]:
items[items['Code'] == '0970367001']

,Code,Name,URL,Img_URL,Img_Ext,Category,Parent_Category,Top_Category,Sub_Category_Name,Parent_Category_Name,Category_Name
14973,0970367001,Cargoshorts Relaxed Fit,/de_de/productpage.0970367001.html,//lp2.hm.com/hmgoepprod?set=source[/f0/41/f041...,.jpg,332.0,243.0,Men,Men_Shorts_Kurze-Jogginghosen,Shorts,Men_Shorts
15011,0970367001,Cargoshorts Relaxed Fit,/de_de/productpage.0970367001.html,//lp2.hm.com/hmgoepprod?set=source[/f0/41/f041...,.jpg,335.0,243.0,Men,Men_Shorts_Cargo-Shorts,Shorts,Men_Shorts
15030,0970367001,Cargoshorts Relaxed Fit,/de_de/productpage.0970367001.html,//lp2.hm.com/hmgoepprod?set=source[/f0/41/f041...,.jpg,336.0,243.0,Men,Men_Shorts_Casual-Shorts,Shorts,Men_Shorts


### Products are repeated because of different sub-categories

### Keeping only relevant columns

In [17]:
items_u = items[['Code', 'Name', 'Img_Ext', 'Parent_Category', 'Top_Category', 'Parent_Category_Name', 'Category_Name']].drop_duplicates()

In [18]:
items_u['Code'].value_counts()

0945068010    3
1003206003    3
1038173002    3
0945068019    3
1005708001    3
             ..
0786187005    1
1055453004    1
1013097003    1
0888448007    1
0824763001    1
Name: Code, Length: 16473, dtype: int64

In [19]:
items_u[items_u['Code'] == '0945068010']

,Code,Name,Img_Ext,Parent_Category,Top_Category,Parent_Category_Name,Category_Name
12269,0945068010,Sportshirt Relaxed Fit,.jpg,226.0,Men,Hoodies & Sweatshirts,Men_Hoodies-&-Sweatshirts
13039,0945068010,Sportshirt Relaxed Fit,.jpg,230.0,Men,T-Shirts,Men_T-Shirts
14710,0945068010,Sportshirt Relaxed Fit,.jpg,241.0,Men,Sportmode,Men_Sportmode


Some items are also there in multiple parent categories - let's drop these products
P.S. - Have checked separately, dropping smaller product categories first (i.e. those with <100 products) is not helping

In [20]:
# let's drop these products and keep only those which have a unique category assigned to them
item_cnts = items_u['Code'].value_counts()
items_to_drop = list(item_cnts[item_cnts > 1].index)

In [21]:
items_u = items_u[~items_u['Code'].isin(items_to_drop)].copy()

In [22]:
items_u.groupby('Category_Name').agg({'Code': ['count', pd.Series.nunique]})

Code        
                            count nunique
Category_Name                            
Baby_Accessoires              123     123
Baby_Kleidung                1058    1058
Baby_Outdoor                   55      55
Baby_Schuhe                    67      67
Kids_Accessoires              286     286
Kids_Kleidung                4237    4237
Kids_Outdoor                  268     268
Kids_Schuhe                   198     198
Men_Accessoires               271     271
Men_Blazer-&-Anzüge            39      39
Men_Cardigans-&-Pullover      130     130
Men_Hemden                    160     160
Men_Hoodies-&-Sweatshirts     143     143
Men_Hosen                     174     174
Men_Jacken-&-Mäntel            98      98
Men_Jeans                      93      93
Men_Nachtwäsche                20      20
Men_Schuhe                    124     124
Men_Shorts                     61      61
Men_Sportmode                  55      55
Men_T-Shirts                  244     244
Men_Unterwäsche                90      90
Women_Accessoires             700     700
Women_Bademode                223     223
Women_Basics                   27      27
Women_Beauty                  669     669
Women_Cardigans-&-Pullover    281     281
Women_Hemden-&-Blusen         373     373
Women_Hoodies-&-Sweatshirts   203     203
Women_Hosen                   545     545
Women_Jacken-&-Mäntel         275     275
Women_Jeans                   466     466
Women_Jumpsuits-&-Playsuits    39      39
Women_Kleider                 646     646
Women_Nachtwäsche             260     260
Women_Röcke                   151     151
Women_Schuhe                  266     266
Women_Shirts-&-Tops           568     568
Women_Shorts                  215     215
Women_Socken-&-Strumpfhosen   126     126
Women_Sportbekleidung         256     256
Women_Unterwäsche             817     817

In [23]:
print(f"Total number of categories: {items['Category_Name'].nunique()}")

Total number of categories: 44


In [24]:
print(f"Total number of categories with more than 100 products: {np.where(items_u.groupby('Category_Name')['Code'].size().values >= 100, 1, 0).sum()}")

Total number of categories with more than 100 products: 31


### Dropping categories with less than 100 products

In [25]:
cnt_of_prods = items_u.groupby('Category_Name')['Code'].count()
cats_to_keep = list(cnt_of_prods[cnt_of_prods >= 100].index)
cats_to_keep

['Baby_Accessoires',
 'Baby_Kleidung',
 'Kids_Accessoires',
 'Kids_Kleidung',
 'Kids_Outdoor',
 'Kids_Schuhe',
 'Men_Accessoires',
 'Men_Cardigans-&-Pullover',
 'Men_Hemden',
 'Men_Hoodies-&-Sweatshirts',
 'Men_Hosen',
 'Men_Schuhe',
 'Men_T-Shirts',
 'Women_Accessoires',
 'Women_Bademode',
 'Women_Beauty',
 'Women_Cardigans-&-Pullover',
 'Women_Hemden-&-Blusen',
 'Women_Hoodies-&-Sweatshirts',
 'Women_Hosen',
 'Women_Jacken-&-Mäntel',
 'Women_Jeans',
 'Women_Kleider',
 'Women_Nachtwäsche',
 'Women_Röcke',
 'Women_Schuhe',
 'Women_Shirts-&-Tops',
 'Women_Shorts',
 'Women_Socken-&-Strumpfhosen',
 'Women_Sportbekleidung',
 'Women_Unterwäsche']

In [26]:
items_u = items_u[items_u['Category_Name'].isin(cats_to_keep)].copy()

In [27]:
items_u.groupby('Category_Name').agg({'Code': ['count', pd.Series.nunique]})

Code        
                            count nunique
Category_Name                            
Baby_Accessoires              123     123
Baby_Kleidung                1058    1058
Kids_Accessoires              286     286
Kids_Kleidung                4237    4237
Kids_Outdoor                  268     268
Kids_Schuhe                   198     198
Men_Accessoires               271     271
Men_Cardigans-&-Pullover      130     130
Men_Hemden                    160     160
Men_Hoodies-&-Sweatshirts     143     143
Men_Hosen                     174     174
Men_Schuhe                    124     124
Men_T-Shirts                  244     244
Women_Accessoires             700     700
Women_Bademode                223     223
Women_Beauty                  669     669
Women_Cardigans-&-Pullover    281     281
Women_Hemden-&-Blusen         373     373
Women_Hoodies-&-Sweatshirts   203     203
Women_Hosen                   545     545
Women_Jacken-&-Mäntel         275     275
Women_Jeans                   466     466
Women_Kleider                 646     646
Women_Nachtwäsche             260     260
Women_Röcke                   151     151
Women_Schuhe                  266     266
Women_Shirts-&-Tops           568     568
Women_Shorts                  215     215
Women_Socken-&-Strumpfhosen   126     126
Women_Sportbekleidung         256     256
Women_Unterwäsche             817     817

In [28]:
items_u.shape

(14456, 7)

In [29]:
print(f"Final dataframe with {items_u.shape[0]:,} products across {items_u['Category_Name'].nunique()} categories")

Final dataframe with 14,456 products across 31 categories


In [30]:
items_u.head()

,Code,Name,Img_Ext,Parent_Category,Top_Category,Parent_Category_Name,Category_Name
0,1071498001,Cut-out-Shirt,.jpg,0.0,Women,Shirts & Tops,Women_Shirts-&-Tops
1,1069183001,Kurzes Shirt mit Cut-out,.jpg,0.0,Women,Shirts & Tops,Women_Shirts-&-Tops
4,1069183002,Kurzes Shirt mit Cut-out,.jpg,0.0,Women,Shirts & Tops,Women_Shirts-&-Tops
5,1021919001,Pullover mit Cut-outs,.jpg,0.0,Women,Shirts & Tops,Women_Shirts-&-Tops
6,1062813001,Tanktop,.jpg,0.0,Women,Shirts & Tops,Women_Shirts-&-Tops


In [31]:
items_u.to_parquet(hnm_file_format.format('Final_Input_Df'))